# Draw Pivot Tree

La funzione **draw_pivot_tree_from_text** prende in input
* una stringa testuale che rappresenta un albero decisionale strutturato tramite indentazione e simboli

e ne costruisce una rappresentazione grafica utilizzando la libreria graphviz. I nodi dell'albero possono essere di due tipi:
* nodi decisionali, che rappresentano condizioni su pivot e medoid, e
* nodi foglia, che riportano un'etichetta, un conteggio e un peso.

La struttura dell'albero viene ricostruita analizzando l'indentazione delle righe e mantenendo la gerarchia tra i nodi grazie a una mappa dei livelli. Il risultato è un oggetto Digraph che può essere visualizzato o esportato per rappresentare graficamente l’albero.

In [1]:
import re
from graphviz import Digraph

def draw_pivot_tree_from_text(tree_str: str):
    lines = tree_str.strip().split('\n')
    dot = Digraph()
    dot.attr('node', shape='box')

    stack = []
    node_id_counter = [0]

    def new_id():
        node_id_counter[0] += 1
        return f"n{node_id_counter[0]}"

    parent_map = {}

    for line in lines:
        indent = len(re.match(r'^\s*', line).group())
        content = line.strip()

        if content.startswith('|--> label:'):
            match = re.match(r'\|--> label: (\d+) \((\d+), ([\d.]+)\)', content)
            if match:
                label, count, weight = match.groups()
                nid = new_id()
                dot.node(nid, f'label: {label}\n({count}, {weight})')
                parent = parent_map.get(indent - 2)
                if parent:
                    dot.edge(parent, nid)
        elif content.startswith('|-+') or content.startswith('-+'):
            match_simple = re.search(r'pivot: (\d+)\s*<=\s*([-\d.]+)', content)
            match_combo = re.search(r'(-?[\d.]+)\s*.*pivot: (\d+)\s*\+\s*([\d.]+)\s*.*medoid: (\d+)\s*<=\s*([-\d.]+)', content)

            nid = new_id()
            if match_combo:
                a1, p1, a2, m2, thres = match_combo.groups()
                expr = f'{a1} * s(x, pivot{p1}) + {a2} * s(x, medoid{m2}) <= {thres}'
            elif match_simple:
                pivot, thres = match_simple.groups()
                expr = f's(x, pivot{pivot}) <= {thres}'
            else:
                expr = content

            dot.node(nid, expr)
            parent = parent_map.get(indent - 2)
            if parent:
                dot.edge(parent, nid)
            parent_map[indent] = nid

    return dot


## Esempio

La **variabile pivot_tree_str** contiene un’istanza di esempio di un albero decisionale basato su pivot, rappresentato in formato testuale. La struttura dell’albero è espressa attraverso l’uso di indentazione e simboli (|-+, |-->) per distinguere i nodi interni *(condizioni decisionali su pivot e medoid)* dalle foglie *(etichette di classificazione con conteggi e pesi)*. Questo esempio viene utilizzato per testare la funzione draw_pivot_tree_from_text, che interpreta il testo e genera un grafo visualizzabile tramite graphviz.

In [ ]:
pivot_tree_str = """|-+ if node_id: 0  pivot: 40 <= 1.58:
  |--> label: 0 (31, 0.3)
  |-+ if node_id: 2  pivot: 73 <= 1.54:
    |--> label: 1 (32, 0.3)
    |-+ if -0.44 node_id: 4  pivot: 4 + 0.58 node_id: 4  medoid: 39 <= -0.38:
      |-+ if node_id: 7  pivot: 47 <= 0.57:
        |--> label: 2 (7, 0.07)
        |--> label: 1 (4, 0.04)
      |--> label: 2 (31, 0.3)"""

dot = draw_pivot_tree_from_text(pivot_tree_str)
dot.render('pivot_tree_parsed', format='png', view=True)
